# Imports

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt
from baa import (
    QuantizedLinearLayerWithActivation,
    replace_linear_layer_with_activation,
    register_linear_layer_forward_hook,
    device_map,
    get_hidden_states_input,
    get_weights,
    add_custom_name_to_linear_layers,
    remove_all_hooks,
    chat_with_model,
    print_memory_usage,
    AccuracyBenchmark,
)
from baa.singletons import hidden_states, names
from datasets import load_dataset
import torch
from dotenv import load_dotenv

load_dotenv()

True

# Load Model

In [2]:
def reload_original_model():
    global model
    global tokenizer
    global model_name
    # model_name = "HuggingFaceTB/SmolLM-135M"
    model_name = "meta-llama/Llama-3.2-1B-Instruct"
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)


reload_original_model()

# Load Benchmark

In [3]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
benchmark = AccuracyBenchmark(model, tokenizer, dataset)

## Run Benchmark on Original Model

In [4]:
print("Original Model Accuracy:", benchmark.evaluate(sample_size=200))

 12%|█▎        | 3/24 [00:09<01:03,  3.04s/it]


KeyboardInterrupt: 

In [7]:
def plot_accuracies(bits, accuracies, title="Accuracy vs Bits"):
    plt.plot(bits, accuracies, "o-")
    plt.xlabel("Bits")
    plt.xticks(bits)
    plt.ylabel("Accuracy")
    plt.ylim(0, 1)  # Set y-axis limits from 0 to 1
    plt.title(title)
    plt.show()

# Full Model Quantization

In [ ]:
def quantize_full_model(bits: int, model=model):
    add_custom_name_to_linear_layers(model)
    replace_linear_layer_with_activation(
        base_model=model,
        quantizer_class=QuantizedLinearLayerWithActivation,
        weight_bits=bits,
        activation_bits=bits,
        exclude_list=[],
        quantized=True,
    )
    torch.cuda.empty_cache()


bit_selection = [2, 3, 4, 5, 6, 8][::-1]
accuracies = []

for bits in bit_selection:
    reload_original_model()
    quantize_full_model(bits, model)
    benchmark.model = model
    with torch.inference_mode():
        accuracy = benchmark.evaluate(sample_size=200)
        accuracies.append(accuracy)
        print(
            "Quantized Model Accuracy with",
            bits,
            "bits:",
            accuracy,
        )

plot_accuracies(bit_selection, accuracies, title="Full Model Quantization")

# Quantize Attention Only

In [ ]:
def quantize_attention(bits: int, model=model):
    add_custom_name_to_linear_layers(model)
    exclude_list = [name for name in names if "self_attn" not in name]
    replace_linear_layer_with_activation(
        base_model=model,
        quantizer_class=QuantizedLinearLayerWithActivation,
        weight_bits=bits,
        activation_bits=bits,
        exclude_list=exclude_list,
        quantized=True,
    )
    torch.cuda.empty_cache()


bit_selection = [2, 3, 4, 5, 6, 8][::-1]
accuracies = []

for bits in bit_selection:
    reload_original_model()
    quantize_attention(bits, model)
    benchmark.model = model
    with torch.inference_mode():
        accuracy = benchmark.evaluate(sample_size=200)
        accuracies.append(accuracy)
        print(
            "Quantized Model Accuracy with",
            bits,
            "bits:",
            accuracy,
        )
plot_accuracies(bit_selection, accuracies, title="Attention Quantization")

# Quantize MLP Only

In [ ]:
def quantize_mlp(bits: int, model=model):
    add_custom_name_to_linear_layers(model)
    exclude_list = [name for name in names if "mlp" not in name]
    replace_linear_layer_with_activation(
        base_model=model,
        quantizer_class=QuantizedLinearLayerWithActivation,
        weight_bits=bits,
        activation_bits=bits,
        exclude_list=exclude_list,
        quantized=True,
    )
    torch.cuda.empty_cache()


bit_selection = [2, 3, 4, 5, 6, 8][::-1]
accuracies = []

for bits in bit_selection:
    reload_original_model()
    quantize_mlp(bits, model)
    benchmark.model = model
    with torch.inference_mode():
        accuracy = benchmark.evaluate(sample_size=200)
        accuracies.append(accuracy)
        print(
            "Quantized Model Accuracy with",
            bits,
            "bits:",
            accuracy,
        )
plot_accuracies(bit_selection, accuracies, title="MLP Quantization")

# Quantize LM Head Only

In [ ]:
def quantize_lm_head(bits: int, model=model):
    add_custom_name_to_linear_layers(model)
    exclude_list = [name for name in names if "lm_head" not in name]
    replace_linear_layer_with_activation(
        base_model=model,
        quantizer_class=QuantizedLinearLayerWithActivation,
        weight_bits=bits,
        activation_bits=bits,
        exclude_list=exclude_list,
        quantized=True,
    )
    torch.cuda.empty_cache()


bit_selection = [2, 3, 4, 5, 6, 8][::-1]
accuracies = []

for bits in bit_selection:
    reload_original_model()
    quantize_lm_head(bits, model)
    benchmark.model = model
    with torch.inference_mode():
        accuracy = benchmark.evaluate(sample_size=200)
        accuracies.append(accuracy)
        print(
            "Quantized Model Accuracy with",
            bits,
            "bits:",
            accuracy,
        )
plot_accuracies(bit_selection, accuracies, title="LM Head Quantization")